**VADER**

In [ ]:
!pip install vaderSentiment

In [ ]:
import pandas as pd
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from sklearn.metrics import accuracy_score

# Load data from CSV file (replace 'reviews.csv' with the actual file path)
df = pd.read_csv('Tweets.csv')
print(df.head())
# Convert any non-string values to strings (if needed)
df['text'] = df['text'].astype(str)

# Initialize the VADER sentiment analyzer
analyzer = SentimentIntensityAnalyzer()

# Define a function to predict sentiment using VADER
def predict_sentiment_vader(text):
    scores = analyzer.polarity_scores(text)
    compound_score = scores['compound']
    return 'positive' if compound_score >= 0.05 else 'negative' if compound_score <= -0.05 else 'neutral'

       textID                                               text  \
0  cb774db0d1                I`d have responded, if I were going   
1  549e992a42      Sooo SAD I will miss you here in San Diego!!!   
2  088c60f138                          my boss is bullying me...   
3  9642c003ef                     what interview! leave me alone   
4  358bd9e861   Sons of ****, why couldn`t they put them on t...   

                         selected_text sentiment  
0  I`d have responded, if I were going   neutral  
1                             Sooo SAD  negative  
2                          bullying me  negative  
3                       leave me alone  negative  
4                        Sons of ****,  negative  


In [ ]:
# Apply the function to get predicted sentiments
df['predicted_sentiment'] = df['text'].apply(predict_sentiment_vader)


In [ ]:
# Calculate the accuracy of the model
accuracy = accuracy_score(df['sentiment'], df['predicted_sentiment'])
print(f'Accuracy of the VADER sentiment analysis model: {accuracy:.2f}')

Accuracy of the VADER sentiment analysis model: 0.64


**TextBlob**

In [ ]:
!pip install textblob

In [ ]:
import pandas as pd
from textblob import TextBlob
from sklearn.metrics import accuracy_score

# Load data from CSV file (replace 'reviews.csv' with the actual file path)
df = pd.read_csv('Tweets.csv')
print(df.head())

# Convert any non-string values to strings (if needed)
df['text'] = df['text'].astype(str)

# Initialize the TextBlob sentiment analyzer
def predict_sentiment_textblob(text):
    analysis = TextBlob(text)
    # TextBlob returns polarity between -1 and 1. Positive sentiment has polarity > 0
    return 'positive' if analysis.sentiment.polarity > 0 else 'negative' if analysis.sentiment.polarity < 0 else 'neutral'

       textID                                               text  \
0  cb774db0d1                I`d have responded, if I were going   
1  549e992a42      Sooo SAD I will miss you here in San Diego!!!   
2  088c60f138                          my boss is bullying me...   
3  9642c003ef                     what interview! leave me alone   
4  358bd9e861   Sons of ****, why couldn`t they put them on t...   

                         selected_text sentiment  
0  I`d have responded, if I were going   neutral  
1                             Sooo SAD  negative  
2                          bullying me  negative  
3                       leave me alone  negative  
4                        Sons of ****,  negative  


In [ ]:
# Apply the function to get predicted sentiments
df['predicted_sentiment'] = df['text'].apply(predict_sentiment_textblob)

In [ ]:
# Calculate the accuracy of the model
accuracy = accuracy_score(df['sentiment'], df['predicted_sentiment'])
print(f'Accuracy of the TextBlob sentiment analysis model: {accuracy:.2f}')

Accuracy of the TextBlob sentiment analysis model: 0.59


**SentiWordNet**

In [ ]:
import pandas as pd
from nltk.corpus import sentiwordnet as swn
from nltk.corpus import wordnet as wn
from nltk import pos_tag, word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.metrics import accuracy_score

# Ensure you have the necessary NLTK data
import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('sentiwordnet')

# Load data from CSV file (replace 'reviews.csv' with the actual file path)
df = pd.read_csv('Tweets.csv')
df['text'] = df['text'].astype(str)
# Initialize WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

# Function to convert NLTK's part of speech tags to WordNet's part of speech tags
def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wn.ADJ
    elif treebank_tag.startswith('V'):
        return wn.VERB
    elif treebank_tag.startswith('N'):
        return wn.NOUN
    elif treebank_tag.startswith('R'):
        return wn.ADV
    else:
        return wn.NOUN


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package sentiwordnet to /root/nltk_data...
[nltk_data]   Package sentiwordnet is already up-to-date!


In [ ]:
# Define a function to predict sentiment using SentiWordNet
def predict_sentiment_sentiwordnet(text):
    sentiment_score = 0.0
    tokens_count = 0

    tagged_sentence = pos_tag(word_tokenize(text))
    for word, tag in tagged_sentence:
        wn_tag = get_wordnet_pos(tag)
        lemma = lemmatizer.lemmatize(word, pos=wn_tag)
        synsets = wn.synsets(lemma, pos=wn_tag)
        if not synsets:
            continue
        # Take the first sense, the most common
        synset = synsets[0]
        swn_synset = swn.senti_synset(synset.name())
        sentiment_score += swn_synset.pos_score() - swn_synset.neg_score()
        tokens_count += 1

    # judgment call – Default to positive or negative
    if not tokens_count:
        return 'neutral'

    # sum greater than 0 => positive sentiment
    if sentiment_score >= 0:
        return 'positive'
    else:
        return 'negative'

In [ ]:
# Apply the function to get predicted sentiments
df['predicted_sentiment'] = df['text'].apply(predict_sentiment_sentiwordnet)

In [ ]:
# Calculate the accuracy of the model
accuracy = accuracy_score(df['sentiment'], df['predicted_sentiment'])
print(f'Accuracy of the SentiWordNet sentiment analysis model: {accuracy:.2f}')

Accuracy of the SentiWordNet sentiment analysis model: 0.42
